In [2]:
import cv2
import numpy as np
import serial
import time

color_bounds = {
    "yellow": ([20, 100, 100], [30, 255, 255]),
    "black": ([0, 0, 0], [180, 255, 30]),
    "blue": ([100, 100, 0], [140, 255, 255]),
    "red": ([0, 100, 100], [10, 255, 255])
}

def detect_color(frame):
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    for color, (lower, upper) in color_bounds.items():
        lower = np.array(lower, dtype=np.uint8)
        upper = np.array(upper, dtype=np.uint8)
        mask = cv2.inRange(hsv_frame, lower, upper)

        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=2)

        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        for contour in contours:
            area = cv2.contourArea(contour)
            x, y, w, h = cv2.boundingRect(contour)
            aspect_ratio = w / float(h)

            if area > 200 and 0.8 <= aspect_ratio <= 1.2:
                return color

    return None

def send_signal_to_arduino(position):
    ser = serial.Serial('COM11', 9600)
    time.sleep(2)

    ser.write(str(position).encode())

    time.sleep(2)

    ser.close()

def main():
    cap = cv2.VideoCapture(1)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        color = detect_color(frame)

        cv2.imshow('Frame', frame)

        if color:
            print(f"Detected color: {color}")
            position = None
            if color == "yellow":
                position = 1
            elif color == "black":
                position = 2
            elif color == "blue":
                position = 3
            elif color == "red":
                position = 4

            if position is not None:
                send_signal_to_arduino(position)
                break

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == '__main__':
    main()


Detected color: black
